#### Chapter Setup

In [ ]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"

## Vanishing/Exploding Gradients Problems

The **Vanishing Gradients Problem** is a side effect of applying the Gradient Descent algorithm to deep neural networks wherein the back propogation error of the lower layers is so low that the algorithm leaves the weights of those layers virtually unchanged throughout the training phase. 

The **Exploding Gradients Problem** is a side effect of applying the Gradient Descent algorithm to deep neural networks wherein the back propogation error of the lower layers is so high that the algorithm updates the weights of those layers so substantially every iteration that the alogirthm diverges.  

This problem can occur due to the difference in the variances of the sigmoid activation function and the normal distribution ($\mu = 0$, $\sigma = 1$) used to initialize layer weights. The use of these two functions causes the inputs of a layer to have lower variance than the output of the same layer. This difference eventually propogates enough that output variance is considerably higher in deeper layers when compared to the variance of the begining layers. This causes the inputs to have exteremly positive or negative values which in turn results in a derivative approcahing 0. Such a low derivative means the weights of a layer will remain unchanged. 

### *Xavier and He Initialization*

For a signal to flow properly through a DNN, the variance of the inputs and outputs of each layer must remain virtually the same going in both directions (forward and back propogation). However this is in practice when the layer does not have an equal amount of input and output connections. Instead the connection weights should be intialized following the *Xavier Initlization* strategy when using a logistic activation function:


$\large \text{Normal Distribution w/ } \mu = 0 \text{ and } \sigma = \sqrt{\frac{2}{n_{inputs} + n_{outputs}}}$ 

OR

$\large \text{Uniform Distribution between } [ -r, +r ] \text{ where } r = \sqrt{\frac{6}{n_{inputs} + n_{outputs}}}$ 

Using this strategy speeds up the training of DNNs. For the ReLU activation function you can use *He activation*. By default the $fully\_connected()$ function uses Xavier initialization.

### *Nonsaturating Activation Function*

The exploding/vanishing gradients problems are in part due to a poor choice in activation function. The ReLU function behaves better in DNNs (as it does not saturate for positive values) when compared to the logistic function but it does have some drawbacks. An example of this is the **dying ReLU** problem wherein some nodes in your DNN simply die and output 0 no matter the input. 

The ReLU function has some variants. The *Leaky ReLU* function ($\text{LeakyReLU}_{\alpha}(z) = \text{max}(\alpha z, \, z)$) uses the hyperparameter $\alpha$ to avoid the problem of dying nodes. However it can result in nodes being in a coma, which essentially means they can die but come back to life if given enough time. 

Another activation function is the *exponential linear unit* (ELU): 

$\large \begin{gather*} \text{ELU}_\alpha (z) =
\begin{cases}
  \alpha \, (e^{\,z} - 1) & \text{ if } z < 0\\    
  z & \text{ if } z \geq 0 \\     
\end{cases}
\end{gather*}$

The ELU function in general has an average output close to 0 (so no vanishing gradients problem), has a nonzero gradient when $z<0$ so nodes won't die, and is smooth everywhere so it speeds up convergence. However it is slower to compute than the ReLU function but this can be mitigated as it converges faster during training.  Both the leaky ReLU and the ELU functions can be used in tensorflow:

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

def leaky_rlu(z, name=None):
    return tf.maximum(0.01*z, z, name=name) 

hidden1 = fully_connected(X, n_hidden1, activation_fn=tf.nn.elu) # ELU
hidden1 = fully_connected(X, n_hidden1, activation_fn=leaky_rlu) # ELU

### *Batch Normalization*

The **Batch Normalization** algorithm consists of adding an operation in the model just before the activation of each layer, simply zero-centering and normalizing the inputs, then scaling and shifting the result using two new parameters per layer. This reduces the chances of the exploding/vanishing gradients problem occuring late into training. The algorithm is given below:

1. $\large \quad \mu_B = \frac{1}{m_b} \, \Sigma_{i=1}^{m_B} \, x^{(i)}$
2. $\large \quad \sigma_B^2 = \frac{1}{m_b} \, \Sigma_{i=1}^{m_B} \, (x^{(i)} - \mu_B)^2$
3. $\large \quad \hat{x}^{(i)} = \frac{x^{(i)} - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$
4. $\large \quad z^{(i)} = \gamma \hat{x}^{(i)} + \beta$

Using Batch Normalization will make your DNN more acurate, make it converge in fewer training iterations, and eliminate the need for regularization. However it make predictions slower, as there are more computations, and it makes your model more complex. 

Here's an implementation of this algorithm in TensorFlow:

In [ ]:
reset_graph()

import tensorflow as tf

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

training = tf.placeholder_with_default(False, shape=(), name='training')

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
bn1_act = tf.nn.elu(bn1)

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = tf.layers.batch_normalization(logits_before_bn, training=training,
                                       momentum=0.9)

The last few lines are very repatative, this can be fixed using Python's partial function:

In [ ]:
from functools import partial

my_batch_norm_layer = partial(tf.layers.batch_normalization,
                              training=training, momentum=0.9)

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
bn1 = my_batch_norm_layer(hidden1)
bn1_act = tf.nn.elu(bn1)
hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = my_batch_norm_layer(hidden2)
bn2_act = tf.nn.elu(bn2)
logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = my_batch_norm_layer(logits_before_bn)

Because of the $training$ variable, when your model is making predictions you should remember to set this parameter to false. 

### *Gradient Clipping*

The **Gradient Clipping** technique is a technique wherein the backpropgation gradient is limited to some max and is *clipped* if the gradient is higher than the max. 

Do do this in tensorflow, you must clip the gradient value before calling $minimize()$ as that function will compute and apply an gradient. Therefore you must ask tensorflow to compute the gradient, clip it, and then apply it to the layer: 

In [ ]:
threshold = 1.0

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(loss) # Compute
capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var) # Clip
              for grad, var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs) # Apply 

## Reusing Pretrained Layers

You should always try to borrow layers from another DNN that tackles a problem similar to yours. This will make training faster as you can fix the weights of those layers and only train the new portion of the DNN. 

### *Reusing a TensorFlow Model*

The following code is an example of reusing only specific portions of a previously trained TensorFlow model:

In [ ]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="hidden[123]") # regular expression
reuse_vars_dict = dict([(var.op.name, var) for var in reuse_vars])
restore_saver = tf.train.Saver(reuse_vars_dict) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):                                      # not shown in the book
        for iteration in range(mnist.train.num_examples // batch_size): # not shown
            X_batch, y_batch = mnist.train.next_batch(batch_size)      # not shown
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})  # not shown
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,  # not shown
                                                y: mnist.test.labels}) # not shown
        print(epoch, "Test accuracy:", accuracy_val)                   # not shown

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

### *Reusing Models from Other Frameworks*

If the model uses another framework then you will have to load the weights into Python and assign to a layer in TensorFlow:

In [ ]:
n_inputs = 2
n_hidden1 = 3

original_w = [...] # Load the weights from the other framework
original_b = [...] # Load the biases from the other framework

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
# [...] Build the rest of the model

# Get a handle on the variables of layer hidden1
with tf.variable_scope("", default_name="", reuse=True):  # root scope
    hidden1_weights = tf.get_variable("hidden1/kernel")
    hidden1_biases = tf.get_variable("hidden1/bias")

# Create dedicated placeholders and assignment nodes
original_weights = tf.placeholder(tf.float32, shape=(n_inputs, n_hidden1))
original_biases = tf.placeholder(tf.float32, shape=n_hidden1)
assign_hidden1_weights = tf.assign(hidden1_weights, original_weights)
assign_hidden1_biases = tf.assign(hidden1_biases, original_biases)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    sess.run(assign_hidden1_weights, feed_dict={original_weights: original_w})
    sess.run(assign_hidden1_biases, feed_dict={original_biases: original_b})
    # [...] Train the model on your new task

### *Freezing the Lower Layers*

To ignore a layer, simply tell the $minimize()$ function which variables to train:

In [ ]:
train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[34]|outputs")
training_op = optimizer.minimize(loss, var_list=train_vars)

### *Tweaking, Dropping, or Replacing the Upper Layers*

* You almost always have to drop the output layer
* Freeze all copied layers, then train your model. If you model underperforms, try un-freezing the topmost frozen layer
* The more data you have the more you can unfreeze

### *Model Zoos*

Model Zoos contain models trained by other people. Look in [TensorFlows model zoo](https://github.com/tensorflow/models) or in [Caffe's model zoo](https://github.com/BVLC/caffe/wiki/Model-Zoo) (look [here](https://github.com/ethereon/caffe-tensorflow) for help converting from Caffe models to tensorflow models) for models to base your projects on. 

### *Unsupervised Pretraining*

You can preform *unsupervised pre-training* if you cannot find a similar model in any model zoo and you don't have much labeled data. Research *Restricted Boltzmann Machines* and *autoencoders* for more information.

### *Pretraining on an Auxillary Task*

Another option is train a network for a related task and re-use the bottom layers. 

If need more labeled training data, gather all your current labeled data and alter in some way to change the classification. This is often faster and cheaper than manually altering data. 

## Fast Optimizers

Four ways to speed up training that we have covered so far:
1. Using a good initailization strategies for the weights
2. Using a good activation function
3. Using Batch Normalization
4. Reusing parts of an old network

All of these methods still use Gradient Descent to change the weights associated with a neuron. In this section we will cover optimizers that are generally faster than Gradient Descent. Having said that, we shall see that the **Adam optimizer** generally works best for training neural networks.

### *Momentum Optimization*

When using a **momentum optimizer**, the current gradient is used in conjunction with a *momentum vector*, $m$, to determine the change in a neuron weights. In physics terms, the current gradient is the acceleration and the momentum vector would be the current velocity. This system introduces a new hyperparameter $\beta$ that acts analogous to friction. 

$\large m \leftarrow \beta m + \eta \nabla_\theta J(\theta)$

$\large \theta \leftarrow \theta - m$

optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)

### *Nesterov Accelerated Gradient*

The **Nesterov Accelerated Gradient** is an improved version of the momentum optimizer that measures the gradient not at the local position but at the position slightly ahead in the direction of the momentum vector. In general, it is much faster than regular momentum optimization.

$\large m \leftarrow \beta m + \eta \nabla_\theta J(\theta + \beta m)$

$\large \theta \leftarrow \theta - m$

In [ ]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9, use_nesterov=True)

### *Ada Grad*

The **Ada Grad optimizer** reduces the learning rate when it encounters steep slopes. It does this in order avoid slowing down in valleys and to converge the global minimum faster. 

$\large s \leftarrow s + \nabla_\theta J(\theta) \, \otimes \, \nabla_\theta J(\theta)$

$\large \theta \leftarrow \theta - \eta \nabla_\theta J(\theta) \, \oslash \, \sqrt{s + \epsilon}$

The first equation accumulates the square of the gradients into the vector $s$. Thus $s_i$ gets larger and larger if $\theta_i$ is steep. The second equation is like gradient descent with a scale factor. 

In general AdaGrad stops too early when training networks so it should not be used to train DNN. 

In [ ]:
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)

### *RMSProp*

The prefered optimizer before Adma optimizers were created, **RMSProp** fixes the early stopping issues of AdaGrad by only considering the last few gradients rather than all of them. 

$\large s \leftarrow \beta s + (1 - \beta) \nabla_\theta J(\theta) \, \otimes \, \nabla_\theta J(\theta)$

$\large \theta \leftarrow \theta - \eta \nabla_\theta J(\theta) \, \oslash \, \sqrt{s + \epsilon}$

In [ ]:
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate, momentum=0.9, decay=0.9, epsilon=1e-10)

### *Adam Optimization*

The **Adaptive Momentum Estimation Optimizer** combines aspects of both Momentum optimization (remebers past gradients) and RMSProp (tracks decaying squared-average of past gradients): 

1. $\large m \leftarrow \beta_1 m + (1 - \beta_1) \nabla_\theta J(\theta)$
2. $\large s \leftarrow \beta_2 s + (1 - \beta_2) \nabla_\theta J(\theta) \, \otimes \, \nabla_\theta J(\theta)$
3. $\large m \leftarrow \frac{m}{1 - \beta_1^T}$
4. $\large s \leftarrow \frac{s}{1 - \beta_2^T}$
5. $\large \theta \leftarrow \theta - \eta m \, \oslash \, \sqrt{s + \epsilon}$

where $T$ is the iteration number starting at 1. 

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

### *Learning Rate Scheduling*

**Learning Schedules** are ways to change the learning rate of an algorithm during training. Here are a few examples:

1. Performance Scheduling: measure the validation error every $N$ steps and reduce the learning rate and reduce the learning rate by $\lambda$ once the error stops dropping
2. Exponential Scheduling: set the learning rate to a function of iteration number $t$: $\eta(t) = \eta_0 10^{-t/r}$. Requires finding the optimal values for the $r$ and $\eta_0$ hypperparameters.

## Avoiding Overfitting Through Regularization

### *Early Stopping*

*Early Stopping*: stop training after performance on validation set set begins to drop. Works good enough in practice by itself but should be used in conjunction with another techinque for best results.

### * $l_1$ and $l_2$ Regularization *

See [here](http://www.chioka.in/differences-between-l1-and-l2-as-loss-function-and-regularization/) for refresher on this subject. $l_1$ regularization can be done as follows in TensorFlow:

In [ ]:
base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")
reg_losses = tf.reduce_sum(tf.abs(W1)) + tf.reduce_sum(tf.abs(W2) + ... + tf.reduce_sum(tf.abs(WN)))
loss = tf.add(base_loss, scale * reg_losses, name="loss")

Or, using TensorFlows bultin features: 

In [ ]:
my_dense_layer = partial(tf.layers.dense, activation=tf.nn.relu, kernel_regularizer=tf.contrib.layers.l1_regularizer(scale))

with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    hidden2 = my_dense_layer(hidden1, n_hidden2, name="hidden2")
    logits = my_dense_layer(hidden2, n_outputs, activation=None, name="outputs")

Don't forget to add your regularized losses!!!

In [ ]:
reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([base_loss] + reg_losses, name="loss")

### *Dropout*

*Dropout*: at every training set every neuron (including input, excluding output) has a probability $p$ of being ignored that step. The $p$ hyperparameter is typically set to 50% and is called the *dropout rate*. This technique can result in an increase of 1% or 2% in accuracy even for state-of-the-art DNNs. 

This technique works becuase neurons now must be as useful as possible on their own as their neighbors could disappear. They cannot rely on just a few input weights, since those may disappear during some iterations, they must instead pay attention to all their inputs. 

Note: after training you must multiply each weight by the *keep probability* $(1-p)$ in order to compensate for the sudden increase of input neurons connected to each neuron. 

In [ ]:
training = tf.placeholder_with_default(False, shape=(), name='training')

dropout_rate = 0.5  # == 1 - keep_prob
X_drop = tf.layers.dropout(X, dropout_rate, training=training)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X_drop, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)
    hidden2 = tf.layers.dense(hidden1_drop, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training)
    logits = tf.layers.dense(hidden2_drop, n_outputs, name="outputs")

### *Max-Norm Regularization*

For each nueron, constrain the weights $w$ of the incoming connections such that $\Vert w \Vert_2 \leq r$ where $r$ is the max-norm hyperparameter and $\Vert \cdot \Vert_2$ is the $l_2$ norm.

### *Data Augmentation*